# Qredo tokenomics simulation - Final params run

### June 2023

## 1. Imports

In [1]:
import os
import pickle
import copy
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
code_dir = os.path.realpath(os.path.join(os.getcwd(), ".."))

import sys
sys.path.append(code_dir)

from mechaqredo.params import default_params_dict
from mechaqredo.generate_scenarios import generate_params_dict_for_scenario
from mechaqredo.data_models import build_model_data_dict_samples
from mechaqredo.sim import run_param_sweep_sim

## 2. Define the parameters

In [3]:
forecast_length = 365*3
data_dict_n_samples = 100
scenarios = ['pessimistic', 'base', 'optimistic']
scenario_tuple_list = list(itertools.product(scenarios, scenarios, scenarios))

file = os.path.realpath("../data/balances.csv")
outfolder = os.path.realpath("../data/sim_data")
wallet_df = pd.read_csv(file)

params_dict = default_params_dict(forecast_length)
params_dict["wallet_balances_vec"] = wallet_df["balance"].values

In [4]:
param_ranges_dict = {
    "tipping_rate": [0.3],
    "protocol_fee_rate": [1],
    "min_stake_amount": [2500],
    "min_stake_duration": [7*4],
    "validator_reward_share": [0.8],
    "staking_rewards_vesting_decay_rate": [np.log(2) / (2.0 * 365)],
    "release_rate_a": [0.8],
    "release_rate_b": [0.5],
    "release_rate_max":[0.0006],
    "ecosystem_refresh_size": [390_000_000],
    "staking_rewards_fund_size": [150_000_000],
}

## 3. Build scenarios and save

In [5]:
scenario_dict_list = []
for token_scenario, usage_scenario, staking_scenario in scenario_tuple_list:
    scenario_params_dict = generate_params_dict_for_scenario(
        params_dict, token_scenario, usage_scenario, staking_scenario, forecast_length
    )
    data_dict_list = build_model_data_dict_samples(
            data_dict_n_samples, forecast_length, scenario_params_dict
        )
    scenario_dict = {
        "token_scenario":token_scenario, 
        "usage_scenario": usage_scenario, 
        "staking_scenario": staking_scenario,
        "params_dict": copy.deepcopy(scenario_params_dict),
        "data_dict_list": copy.deepcopy(data_dict_list),
    }
    scenario_dict_list.append(scenario_dict)
output_file = os.path.join(outfolder, f"final_params_scenarios.pickle")
with open(output_file, "wb") as fp:
    pickle.dump(scenario_dict_list, fp)

## 4. Run simulation

In [6]:
sim_df_list = []
for sim, scenario_dict in enumerate(scenario_dict_list):
    params_dict = scenario_dict["params_dict"]
    data_dict_list = scenario_dict["data_dict_list"]
    sim_df = run_param_sweep_sim(
        forecast_length,
        params_dict,
        param_ranges_dict,
        data_dict_list=data_dict_list,
    )
    sim_df['token_scenario'] = scenario_dict['token_scenario']
    sim_df['usage_scenario'] = scenario_dict['usage_scenario']
    sim_df['staking_scenario'] = scenario_dict['staking_scenario']
    sim_df_list.append(sim_df)

final_sim_df = pd.concat(sim_df_list, ignore_index=True)
sim_file = os.path.join(outfolder, "final_sim_df.pickle")
with open(sim_file, "wb") as fp:
    pickle.dump(final_sim_df, fp)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.21it/s]
